In [1]:
import re
import warnings
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import tableone
from utils import quick_load_liuzhong_health_check_data

%matplotlib inline

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style("white", {"font.sans-serif": ["SimHei"]})

from utils import extend_table1plus_data, get_df_3periods
warnings.filterwarnings("ignore")

df_table1plus, l_consecute_man2p, l_consecute_man3p, df_table1plus_3p_revMM, df_meta_groups, rename_dict = quick_load_liuzhong_health_check_data()
workdir = "/cluster/home/bqhu_jh/projects/healthman/analysis/liuzhong/human/clinical/"

In [2]:
l_items = [
    '心电',
    '心脑血管',
    '影像-内脏',
    '影像-肺',
    '生化-肝脏-酶',
    '甲状腺',
    '血常规-百分比',
    '血常规-类群',
    '血常规-红细胞',
    '血常规-血小板',
    '血常规-计数',
    '高血糖',
    '高血脂'
]

ss = sns.color_palette("hls", len(df_meta_groups["combined_item_name"].unique()))
lut = dict(zip(df_meta_groups["combined_item_name"].unique(), ss.as_hex()))
row_colors = df_meta_groups["combined_item_name"].map(lut)
row_colors.index = df_meta_groups["item_name_en"]
row_colors

item_name_en
HR              #db5f57
HCY             #db5f57
SBP             #db5f57
DBP             #db5f57
HEART.Normal    #db7e57
                 ...   
TPOAb           #db578e
StCOLOR         #db576f
StRBC           #db576f
UNDIG           #db576f
StWBC           #db576f
Name: combined_item_name, Length: 119, dtype: object

In [3]:
l_high_lighted = list(df_meta_groups["item_id"])
l_columns = list(filter(lambda x: x[0]=="v", l_high_lighted))
l_category = list(filter(lambda x: x[0]!="v", l_high_lighted))
l_columns

['v000763',
 'v9319',
 'v000762',
 'v002161',
 'v0009',
 'v0008',
 'v0010',
 'v1235',
 'v0023',
 'v1970',
 'v1164',
 'v0972',
 'v9046',
 'v9047',
 'v9048',
 'v9049',
 'v9050',
 'v9057',
 'v9064',
 'v9045',
 'v9058',
 'v9059',
 'v9060',
 'v9061',
 'v9062',
 'v9063',
 'v9065',
 'v9066',
 'v9051',
 'v9055',
 'v9089',
 'v9090',
 'v9091',
 'v0017',
 'v0045',
 'v0062',
 'v0022',
 'v1162',
 'v1940',
 'v1925',
 'v0015',
 'v0016',
 'v0027',
 'v0026',
 'v0032',
 'v9458',
 'v9459',
 'v9460',
 'v0044',
 'v0001',
 'v0002',
 'v0003',
 'v0004',
 'v0005',
 'v0007',
 'v0018',
 'v0019',
 'v9153',
 'v002043',
 'v002288',
 'v002289',
 'v002401',
 'v0068',
 'v0083',
 'v0282',
 'v0388',
 'v0387',
 'v0378',
 'v0389',
 'v0390',
 'v0081',
 'v0391',
 'v0033',
 'v0723',
 'v0726',
 'v0728',
 'v0729',
 'v3179',
 'v9900',
 'v3029',
 'v4589',
 'v4590',
 'v9907',
 'v9910',
 'v0820',
 'v9909',
 'v0819',
 'v9911',
 'v9908',
 'v3022',
 'v3070',
 'v9453',
 'v0955',
 'v9461',
 'v9462',
 'v9464',
 'v5051',
 'v5054',
 'v505

In [4]:
mytable = tableone.TableOne(df_table1plus[df_table1plus["month"].isin([12,1,2])], l_columns, [], "year-month")

In [5]:
col_order = ['2020-12', '2021-12', '2022-12', 
             '2021-01', '2022-01', '2023-01', 
             '2021-02', '2022-02', '2023-02', 'Missing', 'Overall']

def show_cont_table(my_table_value):
    my_table_value = mytable.cont_table
    my_table_value.loc[l_columns].columns
    my_table_value = my_table_value.loc[l_columns, col_order]
    my_table_value.index = df_meta_groups.loc[l_columns]["item_name"]
    return my_table_value

mytable = tableone.TableOne(df_table1plus[df_table1plus["month"].isin([12,1,2])], l_columns, [], "year-month")
show_cont_table(mytable)

,2020-12,2021-12,2022-12,2021-01,2022-01,2023-01,2021-02,2022-02,2023-02,Missing,Overall
item_name,,,,,,,,,,,
心率,83.7 (13.4),84.4 (13.1),85.6 (13.3),84.9 (13.0),84.8 (12.4),86.8 (13.2),84.3 (12.9),85.3 (12.4),85.0 (12.2),4093,84.8 (12.9)
同型半胱氨酸,11.9 (6.7),9.9 (4.6),10.1 (4.3),9.0 (4.5),9.2 (4.0),9.7 (2.9),8.2 (3.9),9.1 (3.7),10.3 (4.2),28037,9.6 (4.9)
收缩压,123.9 (16.5),124.5 (17.0),124.0 (17.0),121.1 (15.7),119.7 (14.6),122.1 (14.4),120.5 (16.3),121.3 (15.6),121.8 (15.6),4098,122.1 (16.0)
舒张压,74.0 (10.7),73.8 (10.9),73.7 (10.8),71.2 (10.2),71.4 (10.0),73.9 (9.6),71.2 (10.4),72.5 (10.5),72.8 (10.3),4095,72.7 (10.5)
甘油三酯,1.4 (1.3),1.4 (1.0),1.4 (1.1),1.1 (0.9),1.1 (0.9),1.2 (0.9),1.2 (1.0),1.3 (1.1),1.3 (1.2),2040,1.3 (1.1)
...,...,...,...,...,...,...,...,...,...,...,...
甲状腺过氧化物酶抗体,22.6 (110.2),26.6 (126.5),27.5 (128.1),37.7 (153.8),36.2 (144.7),26.5 (107.1),41.1 (154.0),34.2 (142.1),43.8 (168.2),28924,32.0 (137.9)
颜色（大便）,0.4 (0.5),0.1 (0.3),0.1 (0.3),0.8 (0.4),0.2 (0.4),0.2 (0.4),0.3 (0.5),0.1 (0.2),0.0 (0.2),29358,0.1 (0.3)
红细胞,1.0 (0.1),0.0 (0.2),0.0 (0.2),0.9 (0.3),0.2 (0.4),0.0 (0.0),0.9 (0.2),0.0 (0.2),0.1 (0.3),29370,0.1 (0.3)


In [6]:
def show_cont_table(my_table_value):
    my_table_value = mytable.cont_table
    my_table_value.loc[l_columns].columns
    my_table_value = my_table_value.loc[l_columns, col_order]
    my_table_value.index = df_meta_groups.loc[l_columns]["item_name"]
    return my_table_value

mytable = tableone.TableOne(df_table1plus_3p_revMM[df_table1plus_3p_revMM["month"].isin([12,1,2])], 
                                l_columns, [], "year-month")
show_cont_table(mytable)

,2020-12,2021-12,2022-12,2021-01,2022-01,2023-01,2021-02,2022-02,2023-02,Missing,Overall
item_name,,,,,,,,,,,
心率,84.3 (13.1),84.9 (13.0),86.1 (13.2),85.4 (12.7),84.7 (11.8),86.5 (12.0),86.0 (12.8),85.5 (12.3),85.4 (11.9),1559,85.5 (12.4)
同型半胱氨酸,10.9 (5.1),9.7 (3.9),10.0 (3.6),8.0 (3.7),8.3 (3.2),8.8 (2.4),8.7 (4.2),8.9 (3.5),10.2 (4.7),13947,8.9 (4.2)
收缩压,127.3 (18.8),125.9 (18.4),127.8 (19.1),119.5 (14.9),119.4 (14.7),121.6 (14.2),120.0 (14.8),119.9 (14.8),120.9 (15.1),1560,121.2 (15.6)
舒张压,74.4 (11.0),73.5 (10.8),74.6 (10.7),71.1 (10.1),71.4 (9.8),73.8 (9.5),71.1 (9.8),71.3 (9.7),72.1 (9.9),1558,72.0 (10.0)
甘油三酯,1.4 (1.1),1.4 (1.4),1.4 (0.7),1.1 (0.8),1.1 (0.8),1.2 (0.8),1.1 (0.9),1.2 (0.9),1.2 (0.9),170,1.2 (0.9)
...,...,...,...,...,...,...,...,...,...,...,...
甲状腺过氧化物酶抗体,28.3 (137.1),18.7 (102.4),18.9 (99.3),43.6 (155.6),37.7 (116.7),41.5 (125.0),48.4 (167.7),55.4 (177.4),67.7 (208.6),16004,41.5 (155.1)
颜色（大便）,0.4 (0.5),0.1 (0.2),0.1 (0.3),0.5 (0.5),0.1 (0.2),0.2 (0.4),0.7 (0.5),0.2 (0.4),0.0 (0.2),14105,0.1 (0.3)
红细胞,1.0 (0.1),0.0 (0.1),0.0 (0.1),0.9 (0.3),0.0 (0.2),0.0 (0.0),0.9 (0.2),0.2 (0.4),0.1 (0.3),14093,0.1 (0.3)


In [7]:
def get_heart_status(x):
    if x["HEART.Normal"] > 0:
        return "Normal"
    
    if x["HEART.SinusBradycardia"] > 0:
        return "SinusBradycardia"

    if x["HEART.SinusTachycardia"] > 0:
        return "SinusTachycardia"
    
    if x["HEART.SinusRhythm"] > 0:
        return "SinusRhythm"
    
    return "Other"

df_table1plus_3p_revMM["HEART.Status"] = df_table1plus_3p_revMM.apply(lambda x: get_heart_status(x), axis=1)
l_category = ['HEART.Status', 'LUNG.GroundGlassOpacity', 'LIVER.Steatosis']

In [8]:
df_table1plus1 = df_table1plus.copy()

l_basics = ["gender", "age", "v000760", "v000761", "v002266"]
l_basics_en = ["gender", "age", "height", "weight", " waist-circumference"]
df_table1plus1["HEART.Status"] = df_table1plus1.apply(lambda x: get_heart_status(x), axis=1)
df_sub = df_table1plus1[#df_table1plus1["month"].isin([11,12,1,2,3,4,5,6])][
     l_basics+["period"] + l_columns+l_category
]

l_cat_en = l_category[0:1] + list(df_meta_groups.loc[l_category[1:]]["item_name_en"])
l_col_en = list(df_meta_groups.loc[l_columns]["item_name_en"])
df_sub.columns = l_basics_en + [ "period"] + l_col_en + l_cat_en

mytable = tableone.TableOne(df_sub, 
                            l_basics_en + l_col_en+l_cat_en,
                            ["gender"]+l_cat_en, 
                            groupby="period", 
                            pval=True, 
                            overall=False, missing=False, pval_adjust="bonferroni")
mytable.to_excel(f'{workdir}/Table1_sheet1_all.xlsx')
mytable.head(60)

Grouped by period                \
                                            Control-2021  Control-2022   
n                                                  52375         50505   
gender, n (%)                   female      26256 (50.1)  25140 (49.8)   
                                male        26119 (49.9)  25365 (50.2)   
age, mean (SD)                               43.1 (14.5)   42.9 (14.1)   
height, mean (SD)                            165.5 (8.3)   165.8 (8.3)   
weight, mean (SD)                            64.3 (12.5)   64.8 (12.6)   
 waist-circumference, mean (SD)              83.7 (10.2)   83.3 (10.2)   
HR, mean (SD)                                81.9 (12.8)   82.3 (12.7)   
HCY, mean (SD)                                 9.8 (5.3)    10.2 (4.8)   
SBP, mean (SD)                              121.9 (17.0)  121.9 (16.9)   
DBP, mean (SD)                               72.4 (11.0)   72.5 (10.9)   
TG, mean (SD)                                  1.4 (1.1)     1.4 (1.2)   
TC, mean (SD)                                  4.6 (0.9)     4.8 (0.9)   
HDL-C, mean (SD)                               1.3 (0.4)     1.4 (0.4)   
LDL-C, mean (SD)                               2.6 (0.7)     2.7 (0.7)   
FPG, mean (SD)                                 5.0 (1.0)     4.9 (1.0)   
HbA1, mean (SD)                                6.9 (0.8)     6.9 (0.8)   
HbA1c, mean (SD)                               5.7 (0.7)     5.7 (0.7)   
Fasting, mean (SD)                            8.1 (10.9)    8.2 (10.9)   
NEUT%, mean (SD)                              56.0 (8.3)    55.6 (8.2)   
LYMPH%, mean (SD)                             34.1 (7.7)    34.7 (7.7)   
MONO%, mean (SD)                               7.1 (1.7)     6.8 (1.6)   
BASO%, mean (SD)                               0.6 (0.3)     0.6 (0.3)   
EOS%, mean (SD)                                2.3 (1.8)     2.3 (1.8)   
RBC, mean (SD)                                 4.8 (0.5)     4.8 (0.5)   
PLT, mean (SD)                              240.2 (57.9)  244.0 (58.1)   
WBC, mean (SD)                                 6.1 (1.6)     6.1 (1.6)   
Hb, mean (SD)                               143.8 (16.3)  143.6 (16.2)   
Hct, mean (SD)                                43.3 (4.3)    43.1 (4.3)   
MCV, mean (SD)                                90.7 (5.2)    90.4 (5.1)   
MCH, mean (SD)                                30.1 (2.2)    30.1 (2.1)   
MCHC, mean (SD)                             331.7 (11.0)  332.9 (10.6)   
RDW, mean (SD)                                12.7 (1.1)    12.7 (1.1)   
MPV, mean (SD)                                10.7 (1.0)    10.6 (1.0)   
PCT, mean (SD)                                 0.3 (0.1)     0.3 (0.1)   
NEUT, mean (SD)                                3.5 (1.2)     3.4 (1.2)   
LYMPH, mean (SD)                               2.0 (0.6)     2.1 (0.7)   
MONO, mean (SD)                                0.4 (0.1)     0.4 (0.1)   
EOS, mean (SD)                                 0.1 (0.1)     0.1 (0.1)   
BASO, mean (SD)                                0.0 (0.0)     0.0 (0.0)   
ALT, mean (SD)                               21.5 (21.3)   21.7 (20.0)   
AST, mean (SD)                               20.3 (11.9)   20.3 (13.5)   
AST/ALT, mean (SD)                             0.7 (nan)     1.1 (0.5)   
ALP, mean (SD)                               70.9 (21.9)   71.1 (21.3)   
ADA, mean (SD)                                 8.7 (2.4)     8.4 (2.3)   
DPP-4, mean (SD)                             79.7 (18.5)   80.5 (18.5)   
α-GAL, mean (SD)                              20.6 (5.3)    20.5 (5.2)   
TBIL, mean (SD)                               11.9 (5.4)    12.0 (5.5)   
DBIL, mean (SD)                                4.2 (1.6)     4.2 (1.8)   
IBIL, mean (SD)                                7.7 (3.9)     7.8 (4.0)   
TBA, mean (SD)                                 4.5 (4.9)     4.7 (4.9)   
Cr, mean (SD)                                72.5 (19.1)   72.2 (19.0)   
PGI, mean (SD)                             

In [9]:
df_table1plus_3p_revMM["HEART.Status"] = df_table1plus_3p_revMM.apply(lambda x: get_heart_status(x), axis=1)
df_sub = df_table1plus_3p_revMM[#df_table1plus_3p_revMM["month"].isin([12,1,2])][
     l_basics+["period"] + l_columns+l_category
]

l_cat_en = l_category[0:1] + list(df_meta_groups.loc[l_category[1:]]["item_name_en"])
l_col_en = list(df_meta_groups.loc[l_columns]["item_name_en"])
df_sub.columns = l_basics_en + [ "period"] + l_col_en + l_cat_en

mytable = tableone.TableOne(df_sub, 
                            l_basics_en + l_col_en+l_cat_en,
                            ["gender"]+l_cat_en, 
                            groupby="period", 
                            pval=True, 
                            overall=False, missing=False, pval_adjust="bonferroni")

mytable.to_excel(f'{workdir}/Table1_sheet2_3periods.xlsx')
mytable.head(60)

Grouped by period                \
                                            Control-2021  Control-2022   
n                                                  21267         21267   
gender, n (%)                   female      11702 (55.0)  11702 (55.0)   
                                male         9565 (45.0)   9565 (45.0)   
age, mean (SD)                               43.3 (14.5)   44.3 (14.5)   
height, mean (SD)                            165.1 (8.2)   165.2 (8.2)   
weight, mean (SD)                            63.3 (12.2)   63.9 (12.3)   
 waist-circumference, mean (SD)               83.8 (9.8)    84.0 (9.8)   
HR, mean (SD)                                82.8 (12.8)   82.8 (12.5)   
HCY, mean (SD)                                 9.4 (4.9)    10.1 (4.4)   
SBP, mean (SD)                              121.6 (16.8)  122.6 (17.1)   
DBP, mean (SD)                               72.1 (10.8)   72.8 (10.8)   
TG, mean (SD)                                  1.3 (1.1)     1.4 (1.1)   
TC, mean (SD)                                  4.6 (0.9)     4.8 (0.9)   
HDL-C, mean (SD)                               1.4 (0.4)     1.4 (0.4)   
LDL-C, mean (SD)                               2.6 (0.7)     2.7 (0.8)   
FPG, mean (SD)                                 5.0 (0.9)     5.0 (1.0)   
HbA1, mean (SD)                                6.9 (0.7)     7.0 (0.8)   
HbA1c, mean (SD)                               5.7 (0.6)     5.7 (0.7)   
Fasting, mean (SD)                            8.3 (10.7)    8.5 (12.7)   
NEUT%, mean (SD)                              55.7 (8.2)    55.1 (8.3)   
LYMPH%, mean (SD)                             34.4 (7.7)    35.2 (7.8)   
MONO%, mean (SD)                               7.0 (1.6)     6.8 (1.6)   
BASO%, mean (SD)                               0.6 (0.3)     0.6 (0.3)   
EOS%, mean (SD)                                2.3 (1.8)     2.3 (1.8)   
RBC, mean (SD)                                 4.8 (0.5)     4.7 (0.5)   
PLT, mean (SD)                              241.0 (57.9)  244.6 (58.2)   
WBC, mean (SD)                                 6.0 (1.5)     6.1 (1.6)   
Hb, mean (SD)                               142.8 (16.1)  142.5 (16.1)   
Hct, mean (SD)                                43.1 (4.3)    42.8 (4.3)   
MCV, mean (SD)                                90.7 (5.2)    90.5 (5.1)   
MCH, mean (SD)                                30.1 (2.2)    30.1 (2.2)   
MCHC, mean (SD)                             331.0 (11.0)  332.2 (10.7)   
RDW, mean (SD)                                12.7 (1.1)    12.7 (1.1)   
MPV, mean (SD)                                10.7 (1.0)    10.6 (1.0)   
PCT, mean (SD)                                 0.3 (0.1)     0.3 (0.1)   
NEUT, mean (SD)                                3.4 (1.2)     3.4 (1.1)   
LYMPH, mean (SD)                               2.0 (0.6)     2.1 (0.8)   
MONO, mean (SD)                                0.4 (0.1)     0.4 (0.1)   
EOS, mean (SD)                                 0.1 (0.1)     0.1 (0.1)   
BASO, mean (SD)                                0.0 (0.0)     0.0 (0.0)   
ALT, mean (SD)                               20.3 (19.8)   20.9 (17.5)   
AST, mean (SD)                               19.9 (13.2)    20.2 (8.6)   
AST/ALT, mean (SD)                             0.7 (nan)     1.1 (0.5)   
ALP, mean (SD)                               70.0 (20.7)   70.4 (21.1)   
ADA, mean (SD)                                 8.5 (2.2)     8.3 (2.2)   
DPP-4, mean (SD)                             78.1 (17.2)   79.7 (18.1)   
α-GAL, mean (SD)                              20.7 (5.2)    20.6 (5.1)   
TBIL, mean (SD)                               11.9 (5.3)    11.8 (5.3)   
DBIL, mean (SD)                                4.2 (1.6)     4.2 (1.6)   
IBIL, mean (SD)                                7.7 (3.8)     7.6 (3.9)   
TBA, mean (SD)                                 4.4 (6.4)     4.7 (5.2)   
Cr, mean (SD)                                71.7 (21.1)   71.3 (19.9)   
PGI, mean (SD)                             

In [10]:
df_table1plus_3p_revMM["HEART.Status"] = df_table1plus_3p_revMM.apply(lambda x: get_heart_status(x), axis=1)
df_sub = df_table1plus_3p_revMM[df_table1plus_3p_revMM["month"].isin([12,1,2])][
     l_basics+["period"] + l_columns+l_category
]

l_cat_en = l_category[0:1] + list(df_meta_groups.loc[l_category[1:]]["item_name_en"])
l_col_en = list(df_meta_groups.loc[l_columns]["item_name_en"])
df_sub.columns = l_basics_en + [ "period"] + l_col_en + l_cat_en

mytable = tableone.TableOne(df_sub, 
                            l_basics_en + l_col_en+l_cat_en,
                            ["gender"]+l_cat_en, 
                            groupby="period", 
                            pval=True, 
                            overall=False, missing=False, pval_adjust="bonferroni")
mytable.to_excel(f'{workdir}/Table1_sheet3_3periods_12_2.xlsx')
mytable.head(60)

Grouped by period                \
                                            Control-2021  Control-2022   
n                                                   6685          6685   
gender, n (%)                   female       4769 (71.3)   4769 (71.3)   
                                male         1916 (28.7)   1916 (28.7)   
age, mean (SD)                               37.6 (13.8)   38.6 (13.8)   
height, mean (SD)                            163.5 (7.7)   163.7 (7.7)   
weight, mean (SD)                            59.6 (11.1)   60.3 (11.3)   
 waist-circumference, mean (SD)              84.7 (10.0)    84.4 (9.9)   
HR, mean (SD)                                85.6 (12.8)   85.2 (12.3)   
HCY, mean (SD)                                 8.7 (4.3)     9.2 (3.7)   
SBP, mean (SD)                              121.0 (15.7)  120.7 (15.5)   
DBP, mean (SD)                               71.6 (10.1)    71.7 (9.9)   
TG, mean (SD)                                  1.1 (0.9)     1.2 (1.0)   
TC, mean (SD)                                  4.5 (0.9)     4.7 (0.9)   
HDL-C, mean (SD)                               1.4 (0.4)     1.5 (0.4)   
LDL-C, mean (SD)                               2.5 (0.7)     2.6 (0.7)   
FPG, mean (SD)                                 4.9 (0.8)     4.9 (0.8)   
HbA1, mean (SD)                                6.8 (0.6)     7.0 (0.9)   
HbA1c, mean (SD)                               5.6 (0.5)     5.7 (0.8)   
Fasting, mean (SD)                             8.3 (5.4)     9.4 (8.1)   
NEUT%, mean (SD)                              55.2 (8.5)    53.9 (8.5)   
LYMPH%, mean (SD)                             35.0 (7.9)    36.3 (8.0)   
MONO%, mean (SD)                               7.1 (1.7)     7.0 (1.6)   
BASO%, mean (SD)                               0.6 (0.3)     0.6 (0.3)   
EOS%, mean (SD)                                2.1 (1.7)     2.2 (1.8)   
RBC, mean (SD)                                 4.7 (0.5)     4.6 (0.5)   
PLT, mean (SD)                              246.8 (57.4)  250.5 (57.3)   
WBC, mean (SD)                                 6.0 (1.5)     6.1 (1.5)   
Hb, mean (SD)                               139.5 (15.2)  138.7 (15.1)   
Hct, mean (SD)                                42.1 (4.0)    41.8 (4.0)   
MCV, mean (SD)                                90.2 (5.1)    90.2 (5.1)   
MCH, mean (SD)                                29.9 (2.2)    29.9 (2.2)   
MCHC, mean (SD)                             331.4 (11.5)  331.5 (10.8)   
RDW, mean (SD)                                12.6 (1.1)    12.6 (1.1)   
MPV, mean (SD)                                10.8 (1.0)    10.7 (1.0)   
PCT, mean (SD)                                 0.3 (0.1)     0.3 (0.1)   
NEUT, mean (SD)                                3.4 (1.2)     3.3 (1.2)   
LYMPH, mean (SD)                               2.1 (0.6)     2.2 (0.6)   
MONO, mean (SD)                                0.4 (0.1)     0.4 (0.1)   
EOS, mean (SD)                                 0.1 (0.1)     0.1 (0.1)   
BASO, mean (SD)                                0.0 (0.0)     0.0 (0.0)   
ALT, mean (SD)                               18.2 (21.4)   19.0 (16.2)   
AST, mean (SD)                               19.6 (16.1)    20.1 (8.2)   
AST/ALT, mean (SD)                             nan (nan)     1.1 (0.5)   
ALP, mean (SD)                               66.4 (19.3)   66.2 (19.3)   
ADA, mean (SD)                                 8.5 (2.1)     8.2 (2.0)   
DPP-4, mean (SD)                             75.7 (15.8)   79.3 (17.5)   
α-GAL, mean (SD)                              20.3 (4.9)    20.4 (4.9)   
TBIL, mean (SD)                               11.1 (5.0)    11.0 (5.0)   
DBIL, mean (SD)                                3.9 (1.5)     4.0 (1.6)   
IBIL, mean (SD)                                7.2 (3.6)     7.0 (3.6)   
TBA, mean (SD)                                4.9 (11.1)     5.1 (9.0)   
Cr, mean (SD)                                67.4 (18.6)   66.1 (19.2)   
PGI, mean (SD)                             